In [ ]:
import numpy as np
import sys
import os
from dataclasses import dataclass
from scipy.optimize import least_squares, minimize
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from typing import Dict, List
import time
from numpy.typing import ArrayLike
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.tire_model.tire_model_utils import *
from Functions.py_functions.constants import *
from Functions.py_functions.car_configuration import Car, to_vel_frame, to_car_frame, clip

In [ ]:
# generate a car object
car = Car()
car.set_tire(H_R20_18X6_7)
car.toe_front = 0
car.toe_rear = 0
car.update_car()

In [ ]:
def calc_sa(carr, V, omega, beta, delta_F, delta_R):
    alpha_F = np.arctan((V * np.sin(-beta) + (omega * carr.a)) / (V * np.cos(-beta))) - delta_F
    alpha_R = np.arctan((V * np.sin(-beta) - (omega * carr.b)) / (V * np.cos(-beta))) - delta_R
    return alpha_F, alpha_R

def calculate_vel_at_tire(carr, V, omega, beta):
    # beta might have to be negated here, im pretty sure it is correct now but im not 100% sure
    # velocity of the car in the velocity reference frame
    # and adjust for the radius of the turn, and the velocity
    v_F_x = V * np.cos(beta)
    v_F_y = V * np.sin(beta) + omega * carr.a
    v_R_x = V * np.cos(beta)
    v_R_y = V * np.sin(beta) - omega * carr.b
    # velocity of the tire in the car reference frame
    # at the radius of gyration of the tire
    v_F = np.sqrt(v_F_x**2 + v_F_y**2)
    v_R = np.sqrt(v_R_x**2 + v_R_y**2)
    return v_F, v_R

def solve_for_yaw_two(carr, ay_targ, v_avg, beta_x, delta_x, mu_corr, delta_r=np.deg2rad(-4), use_drag=True, vecs=False, sr_lim=0.2):
    delta_r = car.front_camber_gain
    yaw_it = 0.0
    ax_c, ay_c = to_car_frame(0.0, ay_targ, beta_x)
    omega = ay_targ / v_avg #Initial yaw rate [rad/s]
    kappax_f, kappax_r = 0, 0
    bruh, long_error, total_fx = 0, 1, 0
    drag = 0
    if use_drag:
        drag = 0.5 * 1.225 * v_avg**2 * carr.cdA
    long_err = 0.001
    lfx = carr.mass * ax_c + drag
    while bruh < 25 and (long_error > long_err):
        sa_f, sa_r = calc_sa(carr, v_avg, omega, beta_x, delta_x, delta_r)
        fzfl, fzfr, fzrl, fzrr, _, _ = carr.find_contact_patch_loads(long_g=ax_c, lat_g=ay_c, vel=v_avg)
        v_f, v_r = calculate_vel_at_tire(carr, v_avg, omega, beta_x)
        fzf = (fzfl + fzfr) / 2
        fzr = (fzrl + fzrr) / 2
        kappax_r, _, _ = carr.s_r_sel(fzr, sa_r, 0.0, v_r, lfx/2, non_driven=False, mu_corr=mu_corr)

        # Generate slip angles for two-track model as a function of
        # beta and delta along with included parameters for toe.
        fyf, fxf, mzf = carr.steady_state_mmd(fzf, sa_f, kappax_f, v_f, 0.0, delta_x, mu=mu_corr)
        fyr, fxr, mzr = carr.steady_state_mmd(fzr, sa_r, kappax_r, v_r, 0.0, delta_r, mu=mu_corr)
        # Normalized yaw moments created by the front axle and rear
        # axle about the CG
                    
        # Looking from the top down, the positive yaw moment is clock wise
        CN_f = fyf * carr.a + mzf
        CN_r = -fyr * carr.b + mzr # (-) to indicate opposing moment to front axle
        CN_total = CN_f + CN_r
        yaw_it = CN_total / carr.izz # yaw accel [rad/s^2]
        total_fy = (fyf + fyr) * 2
        ay_it = total_fy / carr.mass # lat accel [m/s^2]
        total_fx = (fxf + fxr)*2 - drag
        ax_it = total_fx / carr.mass # lat accel [m/s^2]
        total_fz = (fzf + fzr) * 2
        total_mz = (mzf + mzr) * 2
        # Recalculate new weight transfer and tire vertical loads
        # due to new lat accel
        ax_v, ay_v = to_vel_frame(ax_it, ay_it, beta_x) # equation 51 and 52 in the patton paper
        long_error = abs(ax_c - ax_it)
        bruh += 1
        lfx += carr.mass * (ax_c - ax_it)
    if vecs:
        return ay_v, yaw_it, ax_v, bruh, [fyf*2, fyr*2, total_fy], [fxf*2, fxr*2, total_fx], [mzf*2, mzr*2, total_mz], [fzfl, fzfr, fzrl, fzrr, total_fz], [sa_f, sa_r, 0.0], [delta_x, delta_r, 0.0], [kappax_f, kappax_r, 0.0]

    return ay_v, yaw_it, ax_v, bruh

def loss_func_three(bd, carr, ay_targ, vel, mu_corr, sr_lim):
    ay, yaw, ax, bruh = solve_for_yaw_two(carr, ay_targ, vel, bd[0], bd[1], mu_corr, sr_lim=sr_lim)
    return (ay - ay_targ)**2 + (yaw/5)**2 + ax**2

def find_skidpad_angles(carr, ay_targ, vel, mu_corr, delta_x=0.0, beta_x=0.0, sr_lim=0.2):
    # iterate until the ay converges
    beta_max = np.deg2rad(30)
    delta_max = np.deg2rad(15)
    del_max, del_min = delta_max, -delta_max
    # del_min = -np.deg2rad(5)
    # res = least_squares(loss_func, [beta_x, delta_x], args=(self, ay_targ, vel, mu_corr, sr_lim,), bounds=((-beta_max, del_min), (beta_max, del_max)), verbose=0)
    res = minimize(loss_func_three, [beta_x, delta_x], args=(carr, ay_targ, vel, mu_corr, sr_lim,), bounds=((-beta_max, beta_max), (del_min, del_max)), options=dict(disp=False), method="Nelder-Mead")
    beta, delta = res.x
    ay, yaw, ax, bruh = solve_for_yaw_two(carr, ay_targ, vel, beta, delta, mu_corr, sr_lim=sr_lim)
    return beta, delta, ay, yaw, ax, bruh

def solve_skidpad_time(carr, radius, mu_corr, vel=5, sr_lim=0.2):
    last_beta, last_delta, last_ay, last_yaw, last_ax, last_bruh = 0, 0, 0.0, 0.0, 0.0, 0.0
    last_vel = vel
    vel_inc = 2
    while True:
        ay_targ = vel**2 / radius * -1 # ay is negative for right hand turn on skidpad in SAE sign convension
        beta, delta, ay, yaw, ax, bruh = find_skidpad_angles(carr, ay_targ, vel, mu_corr, beta_x=last_beta, delta_x=last_delta, sr_lim=sr_lim)
        if ay == np.inf or abs(ay - ay_targ) > 0.01 or bruh == 25:
            if vel_inc < 0.001:
                break
            vel_inc /= 2
            vel = last_vel + vel_inc
            continue
        last_beta, last_delta, last_ay, last_yaw, last_ax, last_bruh = beta, delta, ay, yaw, ax, bruh
        last_vel = vel
        vel += vel_inc

    lap_time = 2 * np.pi * radius / last_vel
    return last_beta, last_delta, last_ay, last_yaw, last_ax, last_bruh, last_vel, lap_time

In [ ]:
skidpad_radius_inner = 15.25 / 2 # m
skidpad_radius_outer = skidpad_radius_inner + 3 # m
skidpad_radius = skidpad_radius_inner + 0.711 # m
print(skidpad_radius)

In [ ]:
# converge on the minimum lap time of the skidpad
kappa_lim = 0.2
beta, delta, ay, yaw, ax, bruh, vel, lap_time = solve_skidpad_time(car, skidpad_radius, 0.65, sr_lim = kappa_lim)

print('Velocity: {:.2f} m/s'.format(vel))
print('Beta: {:.2f} deg'.format(np.rad2deg(beta)))
print('Delta: {:.2f} deg'.format(np.rad2deg(delta)))
print('ay: {:.2f} m/s^2'.format(ay))
print('yaw: {:.2f} rad/s^2'.format(yaw))
print('ax: {:.2f} m/s^2'.format(ax))
print('bruh: {}'.format(bruh))
print('Lap time: {:.4f} s'.format(lap_time))
# omega based on lateral acceleration
omega = vel / skidpad_radius
print('Omega: {:.2f} rad/s'.format(omega))

In [ ]:
# make a plot that shows the force and torque vectors at each tire, the car's center of mass, and the center of the skidpad
fig = go.Figure()
# add the skidpad as a circle
fig.add_shape(type="circle", xref="x", yref="y", x0=-skidpad_radius_inner, y0=-skidpad_radius_inner, x1=skidpad_radius_inner, y1=skidpad_radius_inner, line_color="Red", line_width=1)
fig.add_shape(type="circle", xref="x", yref="y", x0=-skidpad_radius, y0=-skidpad_radius, x1=skidpad_radius, y1=skidpad_radius, line_color="LightSeaGreen", line_width=2)
fig.add_shape(type="circle", xref="x", yref="y", x0=-skidpad_radius_outer, y0=-skidpad_radius_outer, x1=skidpad_radius_outer, y1=skidpad_radius_outer, line_color="Red", line_width=1)
# add a point at the center of the skidpad
fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', marker=dict(color='LightSeaGreen', size=5), showlegend=False))
# add the car's center of mass
fig.add_trace(go.Scatter(x=[-skidpad_radius], y=[0], mode='markers', marker=dict(color='red', size=5), showlegend=False))
# find coordinates of the tires
tire_xy_car = np.array([[0.0, car.a], [0.0, -car.b], [0, 0]])
# rotate the tire coordinates to the car's orientation
rot = np.array([[np.cos(-beta), -np.sin(-beta)], [np.sin(-beta), np.cos(-beta)]]) # were rotating it back to world coordinates so we need to negate the angle
tire_xy_vel = np.matmul(rot, tire_xy_car.T).T
# shift the tire coordinates to the car's position
tire_xy_vel[:, 0] -= skidpad_radius
# add the tire coordinates to the plot
# fig.add_trace(go.Scatter(x=tire_xy_vel[:, 0], y=tire_xy_vel[:, 1], mode='markers', marker=dict(color='blue', size=2), showlegend=False))
# get the force and torque vectors at each tire
ay_v, yaw_it, ax_v, bruh, fy_s, fx_s, mz_s, fz_s, sa_s, delta_s, kappa_s = solve_for_yaw_two(car, ay, vel, beta, delta, mu_corr=0.65, vecs=True, sr_lim = kappa_lim)
scalar = 800
# add the force and torque vectors to the plot
names = ['F', 'R', 'CM']
def rot_vec(point, rot):
    pt = np.matmul(rot, point.T).T
    return np.array([[0, 0], pt])
for i in range(3):
    t_rot = np.array([[np.cos(-delta_s[i]), -np.sin(-delta_s[i])], [np.sin(-delta_s[i]), np.cos(-delta_s[i])]])
    f_tot = rot_vec(np.array([-fy_s[i] / scalar, fx_s[i] / scalar]), t_rot)
    f_x = rot_vec(np.array([0, fx_s[i] / scalar]), t_rot)
    f_y = rot_vec(np.array([-fy_s[i] / scalar, 0]), t_rot)
    f_tot = np.matmul(rot, f_tot.T).T + tire_xy_vel[i]
    f_x = np.matmul(rot, f_x.T).T + tire_xy_vel[i]
    f_y = np.matmul(rot, f_y.T).T + tire_xy_vel[i]
    name = names[i]
    fig.add_trace(go.Scatter(x=f_tot[:, 0], y=f_tot[:, 1], legendgroup=f"{name}", name=name, mode='lines', line=dict(color='green', width=2), hovertext='{}: {:.2f} N'.format(name, np.sqrt(fx_s[i]**2 + fy_s[i]**2))))
    fig.add_trace(go.Scatter(x=f_x[:, 0], y=f_x[:, 1], legendgroup=f"{name}", name=name, mode='lines', line=dict(color='pink', width=2), hovertext='{} Fx: {:.2f} N'.format(name, fx_s[i]), showlegend=False))
    fig.add_trace(go.Scatter(x=f_y[:, 0], y=f_y[:, 1], legendgroup=f"{name}", name=name, mode='lines', line=dict(color='pink', width=2), hovertext='{} Fy: {:.2f} N'.format(name, fy_s[i]), showlegend=False))
# add the car's orientation
# fig.add_trace(go.Scatter(x=[skidpad_radius, skidpad_radius + np.sin(beta)], y=[0, np.cos(beta)], mode='lines', line=dict(color='red', width=2)))
print(f"Fz: {fz_s}")
print(f"SA: {np.rad2deg(np.array(sa_s))}")
print(f"Delta: {np.rad2deg(np.array(delta_s))}")
print(f"SL: {kappa_s}")
print(ay_v, ax_v)

fig.update_layout(title='Skidpad Forces and Torques', xaxis_title='x (m)', yaxis_title='y (m)', width=1000, height=1000, template="plotly_dark")
fig.update_xaxes(scaleanchor="y", scaleratio=1)
fig.show()

In [ ]:
lenz = 50
toe_range = np.deg2rad(np.linspace(-10, 10, lenz))
beta, delta, ay, yaw, ax, bruh, vel, lap_time = np.zeros(lenz), np.zeros(lenz), np.zeros(lenz), np.zeros(lenz), np.zeros(lenz), np.zeros(lenz), np.zeros(lenz), np.zeros(lenz)
sa_f, sa_r, kappa_f, kappa_r = np.zeros(lenz), np.zeros(lenz), np.zeros(lenz), np.zeros(lenz)
for ind, toe in enumerate(toe_range):
    car.front_camber_gain = toe
    beta[ind], delta[ind], ay[ind], yaw[ind], ax[ind], bruh[ind], vel[ind], lap_time[ind] = solve_skidpad_time(car, skidpad_radius, 0.65, sr_lim = kappa_lim)
    ay_v, yaw_it, ax_v, bruh_s, fy_s, fx_s, mz_s, fz_s, sa_s, delta_s, kappa_s = solve_for_yaw_two(car, ay[ind], vel[ind], beta[ind], delta[ind], mu_corr=0.65, vecs=True, sr_lim = kappa_lim)
    sa_f[ind], sa_r[ind], kappa_f[ind], kappa_r[ind] = sa_s[0], sa_s[1], kappa_s[0], kappa_s[1]

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.rad2deg(toe_range), y=vel, mode='lines', line=dict(color='green', width=2), name='Velocity'))
fig.add_trace(go.Scatter(x=np.rad2deg(toe_range), y=ay, mode='lines', line=dict(color='red', width=2), name='Lateral Acceleration'))
fig.update_layout(title='Skidpad Velocity and Lateral Acceleration vs Toe Angle', xaxis_title='Toe Angle (deg)', yaxis_title='Velocity (m/s) and Lateral Acceleration (m/s^2)', width=1000, height=1000, template="plotly_dark")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.rad2deg(toe_range), y=np.rad2deg(beta), mode='lines', line=dict(color='green', width=2), name='Beta'))
fig.add_trace(go.Scatter(x=np.rad2deg(toe_range), y=np.rad2deg(delta), mode='lines', line=dict(color='red', width=2), name='Delta'))
fig.add_trace(go.Scatter(x=np.rad2deg(toe_range), y=np.rad2deg(sa_f), mode='lines', line=dict(color='blue', width=2), name='SA_F'))
fig.add_trace(go.Scatter(x=np.rad2deg(toe_range), y=np.rad2deg(sa_r), mode='lines', line=dict(color='green', width=2), name='SA_R'))
fig.update_layout(title='Skidpad Beta and Delta vs Toe Angle', xaxis_title='Toe Angle (deg)', yaxis_title='Beta (deg) and Delta (deg)', width=1000, height=1000, template="plotly_dark")
fig.show()